In [1]:
# from minicons import scorer
from transformers import set_seed
from transformers import AutoTokenizer, AutoModelForCausalLM

/scratch/miniconda3/envs/dev/lib/python3.11/site-packages/transformers/utils/hub.py:128: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


In [2]:
model_name = "meta-llama/Meta-Llama-3-8B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)
model.to("cuda:0")

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((4096,), eps=1e-05)
    (rotary_

In [2]:
# lm = scorer.IncrementalLMScorer("gpt2-xl", device="cuda:0")

In [41]:
s_full = "The nurse, who likes french cuisine, adopted a rescue dog."
s_vp1 = "The nurse likes french cuisine."
s_vp2 = "The nurse adopted a rescue dog."


def prompt(sentence):
    return f'Ellie said, "{sentence}", and Marco replied, "'


def chat_prompt(sentence, tok=tokenizer):
    return tok.apply_chat_template(
        [   {"role": "system", "content": "Please respond to the following message as naturally as possible."},
            { "role": "user", "content": sentence}],
        tokenize=False,
        add_generation_prompt=True,
    )

In [42]:
# encoded = lm.tokenizer(s_vp1, return_tensors="pt")
# encoded = encoded.to(lm.device)

encoded = tokenizer(chat_prompt(s_vp1), return_tensors="pt")
encoded = encoded.to("cuda:0")

In [43]:
set_seed(1024)

generations = model.generate(
    **encoded, 
    num_return_sequences=10, 
    max_length=50, 
    do_sample=True, 
    top_p=0.8,
    temperature=0.9,
    # typical_p=0.9,
    # min_p=0.2,
    # top_p=0.8,
    # top_k = 5000,
    # force_words_ids=[[857]],
    stop_strings=["."],
    tokenizer=tokenizer
)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


In [44]:
tokenizer.batch_decode(generations,skip_special_tokens=True)

["system\n\nPlease respond to the following message as naturally as possible.user\n\nThe nurse likes french cuisine.assistant\n\nThat's great! French cuisine is renowned for its rich flavors and intricate preparations.",
 "system\n\nPlease respond to the following message as naturally as possible.user\n\nThe nurse likes french cuisine.assistant\n\nThat's interesting! I've always loved the rich flavors and aromas of French cuisine",
 "system\n\nPlease respond to the following message as naturally as possible.user\n\nThe nurse likes french cuisine.assistant\n\nThat's great! French cuisine is renowned for its rich flavors and intricate preparations.",
 "system\n\nPlease respond to the following message as naturally as possible.user\n\nThe nurse likes french cuisine.assistant\n\nThat's great! French cuisine is known for its rich flavors and intricate preparations.",
 "system\n\nPlease respond to the following message as naturally as possible.user\n\nThe nurse likes french cuisine.assistant

In [32]:
tokenizer("she does")

{'input_ids': [7091, 857], 'attention_mask': [1, 1]}